In [19]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [20]:
features = [
    "temp2_max(c)", "temp2_min(c)", "temp2_ave(c)", 
    "suface_pressure(pa)", "wind_speed50_max(m/s)", "wind_speed50_min(m/s)", 
    "wind_speed50_ave(m/s)", "prectotcorr", "total_demand(mw)"
]

In [27]:
data = pd.read_csv(r"C:\Users\Idea\Documents\Programming languages\Python\Notebooks\Time Series\Project_1\Data\data.csv", index_col="date", parse_dates=True)

In [28]:
data.head()

,temp2_max(c),temp2_min(c),temp2_ave(c),suface_pressure(pa),wind_speed50_max(m/s),wind_speed50_min(m/s),wind_speed50_ave(m/s),prectotcorr,total_demand(mw),max_generation(mw)
date,,,,,,,,,,
2018-01-01,24.48,13.78,19.130,101.08,5.05,0.23,2.640,0.00,8000.0,7651.0
2018-01-02,23.16,15.28,19.220,100.94,6.20,1.59,3.895,0.01,7900.0,7782.0
2018-01-03,22.65,11.52,17.085,101.12,6.96,3.64,5.300,0.17,7900.0,7707.0
2018-01-04,20.32,8.19,14.255,101.34,5.91,1.54,3.725,0.34,8000.0,7715.0
2018-01-05,20.79,8.24,14.515,101.17,6.02,3.09,4.555,0.00,7200.0,6926.0


In [22]:
lag = 1

In [16]:
# Create a new dataframe to store predictions for the next 365 days
future_predictions = pd.DataFrame(index=pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=365, freq='D'))

scalers_dir = r"C:\Users\Idea\Documents\Programming languages\Python\Notebooks\Time Series\Project_1\Notebooks\scalers"

model_dir = r"C:\Users\Idea\Documents\Programming languages\Python\Notebooks\Time Series\Project_1\Notebooks\models"


# Loop through each feature
for feature in features:
    print(f"Predicting future values for feature: {feature}")
    
    # Load the saved scaler and model
    # Load the pre-fitted scaler for the current feature
    scaler_path = os.path.join(scalers_dir, f"{feature.replace('/', '')}_scaler.pkl")
    with open(scaler_path, 'rb') as f:
        scaler = joblib.load(f)

    model_path = os.path.join(model_dir, f"{feature.replace('/', '')}_LSTM.h5")
    model = load_model(model_path, custom_objects={'mse': mean_squared_error})
    
    # Get the last lagged values from the test data
    last_values = data[[feature]].iloc[-lag:].values
    scaled_last_values = scaler.transform(last_values)
    
    # Predict the next 365 days
    predictions = []
    for _ in range(365):
        # Reshape the input to match the model's expected input shape
        input_data = scaled_last_values[-lag:].reshape((1, lag, 1))
        
        # Predict the next value
        next_prediction = model.predict(input_data, verbose=0)
        
        # Inverse transform the prediction to get the original scale
        next_prediction_original = scaler.inverse_transform(next_prediction)
        predictions.append(next_prediction_original[0, 0])
        
        # Append the prediction to the scaled_last_values for the next iteration
        scaled_last_values = np.append(scaled_last_values, next_prediction, axis=0)
    
    # Add the predictions to the future_predictions dataframe
    future_predictions[feature] = predictions

Predicting future values for feature: temp2_max(c)


Predicting future values for feature: temp2_min(c)


Predicting future values for feature: temp2_ave(c)


Predicting future values for feature: suface_pressure(pa)


Predicting future values for feature: wind_speed50_max(m/s)


Predicting future values for feature: wind_speed50_min(m/s)


Predicting future values for feature: wind_speed50_ave(m/s)


Predicting future values for feature: prectotcorr


Predicting future values for feature: total_demand(mw)
Future predictions saved to 'future_predictions.csv'


In [36]:
# Load the saved model
model = load_model(r"C:\Users\Idea\Documents\Programming languages\Python\Notebooks\Time Series\Project_1\Notebooks\models\final_model 291.19.h5", custom_objects={'mse': mean_squared_error})

# Prepare the test data
y_test = data["max_generation(mw)"]
X_test = data.drop(columns=["total_demand(mw)"])

# Fit the scalers on the training data (you need to ensure the same scaling as the original notebook)
scaler_X=joblib.load("X_train_scaler.pkl")
scaler_y=joblib.load("y_train_scaler.pkl")

X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Reshape X_test_scaled to match the expected input shape
n_input = 1  # Same as defined in the original notebook
n_features = X_test.shape[1]
X_test_scaled_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], n_input, n_features))

# Predict and inverse transform the predictions
y_pred_scaled = model.predict(X_test_scaled_reshaped[-365:], verbose=0)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

In [37]:
len(future_predictions)

365

In [38]:
len(y_pred)

365

In [39]:
future_predictions["max_generation(mw)"] = y_pred

In [40]:
# Save the future predictions to a new dataframe
future_predictions.to_csv("future_predictions.csv", index=True)
print("Future predictions saved to 'future_predictions.csv'")

Future predictions saved to 'future_predictions.csv'
